In [1]:
import pandas as pd
import math

In [2]:
# Load crime data and take a look at it
columns = ["CASE#", " PRIMARY DESCRIPTION", "LATITUDE", "LONGITUDE"]
crime = pd.read_csv("Crimes_Data.csv", usecols = columns)
crime = crime.dropna()
crime.head()

,CASE#,PRIMARY DESCRIPTION,LATITUDE,LONGITUDE
2,JE266628,THEFT,41.748486,-87.602675
3,JE266536,ASSAULT,41.880661,-87.731186
5,JE267466,ASSAULT,41.871540,-87.705839
6,JE266473,HOMICIDE,41.780851,-87.649674
7,JE267222,OTHER OFFENSE,41.859989,-87.735995


In [3]:
# Type of crime descriptive statics
crime[" PRIMARY DESCRIPTION"].describe()

count      202883
unique         31
top       BATTERY
freq        40556
Name:  PRIMARY DESCRIPTION, dtype: object

In [4]:
# Total types of crimes
set(crime[" PRIMARY DESCRIPTION"])

{'ARSON',
 'ASSAULT',
 'BATTERY',
 'BURGLARY',
 'CONCEALED CARRY LICENSE VIOLATION',
 'CRIMINAL DAMAGE',
 'CRIMINAL SEXUAL ASSAULT',
 'CRIMINAL TRESPASS',
 'DECEPTIVE PRACTICE',
 'GAMBLING',
 'HOMICIDE',
 'HUMAN TRAFFICKING',
 'INTERFERENCE WITH PUBLIC OFFICER',
 'INTIMIDATION',
 'KIDNAPPING',
 'LIQUOR LAW VIOLATION',
 'MOTOR VEHICLE THEFT',
 'NARCOTICS',
 'NON-CRIMINAL',
 'OBSCENITY',
 'OFFENSE INVOLVING CHILDREN',
 'OTHER NARCOTIC VIOLATION',
 'OTHER OFFENSE',
 'PROSTITUTION',
 'PUBLIC INDECENCY',
 'PUBLIC PEACE VIOLATION',
 'ROBBERY',
 'SEX OFFENSE',
 'STALKING',
 'THEFT',
 'WEAPONS VIOLATION'}

In [5]:
# A dictionary that maps the crime types to Cambridge CHI (Cambridge Crime Harm Index) crime types
CRIME_MAPPING = {
    'ARSON': 'Damage (Arson)',
    'ASSAULT': 'Recent Sexual Assault',
    'BATTERY': 'Recent Sexual Assault',
    'BURGLARY': 'Dwelling Burglary',
    'CONCEALED CARRY LICENSE VIOLATION': 'Threatening Behaviour',
    'CRIMINAL DAMAGE': 'Damage (Property)',
    'CRIMINAL SEXUAL ASSAULT': 'Recent Sexual Assault',
    'CRIMINAL TRESPASS': 'Threatening Behaviour',
    'DECEPTIVE PRACTICE': 'Damage (Property)',
    'GAMBLING': 'Non Crime Incident',
    'HOMICIDE': 'Murder',
    'HUMAN TRAFFICKING': 'Deprivation of Liberty',
    'INTERFERENCE WITH PUBLIC OFFICER': 'Threatening Behaviour',
    'INTIMIDATION': 'Threatening Behaviour',
    'KIDNAPPING': 'Attempted Murder',
    'LIQUOR LAW VIOLATION': 'Non Crime Incident',
    'MOTOR VEHICLE THEFT': 'Motor Vehicle Theft',
    'NARCOTICS': 'Manslaughter',
    'NON-CRIMINAL': 'Non Crime Incident',
    'OBSCENITY': 'Non Crime Incident',
    'OFFENSE INVOLVING CHILDREN': 'Threatening Behaviour',
    'OTHER NARCOTIC VIOLATION': 'Manslaughter',
    'OTHER OFFENSE': 'Non Crime Incident',
    'PROSTITUTION': 'Non Crime Incident',
    'PUBLIC INDECENCY': 'Non Crime Incident',
    'PUBLIC PEACE VIOLATION': 'Domestic Assault',
    'ROBBERY': 'Robbery (Business)',
    'SEX OFFENSE': 'Recent Sexual Assault',
    'STALKING': 'Recent Sexual Assault',
    'THEFT': 'Theft',
    'WEAPONS VIOLATION': 'Attempted Murder'
}

# CHI Scores dictionary
CHI_SCORES = {
    'Murder': 5475,
    'Manslaughter': 3825,
    'Attempted Murder': 1460,
    'Robbery (Business)': 365,
    'Recent Sexual Assault': 365,
    'Robbery (Non-Business)': 365,
    'Historical Sexual Assault': 365,
    'Deprivation of Liberty': 84,
    'Damage (Arson)': 33,
    'Domestic Assault': 20,
    'Dwelling Burglary': 20,
    'Non-Dwelling Burglary': 20,
    'Motor Vehicle Theft': 20,
    'Threatening Behaviour': 10,
    'Damage (Property)': 2,
    'Theft': 2,
    'Non-Crime Incident': 0.1
}

In [6]:
# Calculate number of crime incidents in the neighborhood
# From website:
# https://www.geeksforgeeks.org/program-distance-two-points-earth/#:~:text=For%20this%20divide%20the%20values,is%20the%20radius%20of%20Earth.
def if_crime_in_neighborhood(la1, lo1, la2, lo2, radius):
    """
    Determine whether a crime case is within the neighborhood
    of a residential property.
    
    Inputs:
        la1: latitude of property
        lo1: longitude of property
        la2: latitude of crime case
        lo2: longtitude of crime case
        radius: radius of neighborhood
        
    Outputs:
        True/False: crime case in/not in neighborhood
    """

    la1 = math.radians(la1)
    lo1 = math.radians(lo1)
    la2 = math.radians(la2)
    lo2 = math.radians(lo2)

    la_dist = la1 - la2
    lo_dist = lo1 - lo2

    a = math.sin(la_dist/2)**2 + math.cos(la1) * math.cos(la2) * math.sin(lo_dist/2)**2

    c = 2 * math.asin(math.sqrt(a))

    earth_r_in_mile = 3956

    if c * earth_r_in_mile <= radius:
        return True
    else:
        return False


In [7]:
housing = pd.read_csv("apartments.csv", index_col="property_name")
housing.head()

,address,latitude,longitude,price_range,bedroom,bathroom,room_area,amenities
property_name,,,,,,,,
The Residences at NewCity,"1457 N Halsted St, Chicago IL 60642",41.90801,-87.64773,"$2,126 - $4,215",Studio - 2 bd,1 - 2 ba,"590 - 1,315 sq ft","Pool, Fitness Center, Elevator, Clubhouse, Con..."
Elevate,"930 W Altgeld St, Chicago IL 60614",41.92760,-87.65243,"$1,910 - $14,435",Studio - 3 bd,1 - 2.5 ba,"464 - 2,616 sq ft","Pool, Fitness Center, Elevator, Clubhouse, In ..."
1407 On Michigan,"1407 S Michigan Ave, Chicago IL 60605",41.86377,-87.62360,"$1,750 - $5,650",Studio - 3 bd,1 - 2 ba,"446 - 1,687 sq ft","Pool, Fitness Center, Elevator, Doorman, Clubh..."
The Pavilion,"5441 NE NEast River Rd, Chicago IL 60656",41.97831,-87.84335,"$1,115 - $2,420",Studio - 4 bd,1 - 3 ba,"502 - 2,317 sq ft","Pool, Fitness Center, Controlled Access, Busin..."
Gild,"1206-1212 N State Pky, Chicago IL 60610",41.90423,-87.62905,"$2,260 - $6,322",Studio - 2 bd,1 - 2 ba,"463 - 1,272 sq ft","Fitness Center, Elevator, Concierge, Clubhouse..."


In [8]:
security = []

for name, info in housing.iterrows():
    crime_in_neighborhood = crime[
        if_crime_in_neighborhood(
            crime['LATITUDE'],
            crime['LONGITUDE'],
            info['latitude'],
            info['longitude'],
            2
        )
    ]

TypeError: cannot convert the series to <class 'float'>

In [ ]:
crime_in_neighborhood = crime[crime.apply(
    if_crime_in_neighborhood(
            crime['LATITUDE'],
            crime['LONGITUDE'],
            41.92760,
            -87.64773,
            2
        ),axis=1
)
    ]

crime_in_neighborhood

TypeError: cannot convert the series to <class 'float'>

In [ ]:
#data visualization
import folium

my_map = folium.Map(
    location=[41.9665, -87.6533],
    zoom_start=10
)

for name, info in housing.iterrows():
    folium.Marker(
        location=[info["latitude"], info["longitude"]],
        popup=folium.Popup(
            pd.DataFrame(housing.loc[name]).drop(["latitude", "longitude"]).to_html(), 
            max_width=300
            ),
        tooltip=name,
    ).add_to(my_map)

my_map

In [ ]:
pd.DataFrame(housing.loc["Elevate"]).drop(["latitude", "longitude"])

,Elevate
address,"930 W Altgeld St, Chicago IL 60614"
price_range,"$1,910 - $14,435"
bedroom,Studio - 3 bd
bathroom,1 - 2.5 ba
room_area,"464 - 2,616 sq ft"
amenities,"Pool, Fitness Center, Elevator, Clubhouse, In ..."
